# Grover search

Based on:
- [Inversion over mean](https://gitlab.com/qworld/bronze-qiskit/-/blob/master/quantum-with-qiskit/Q84_Inversion_About_the_Mean.ipynb) QBronze.
- [Grover search](https://gitlab.com/qworld/bronze-qiskit/-/blob/master/quantum-with-qiskit/Q92_Grovers_Search_Implementation.ipynb) QBronze.
- [Wiki](https://en.wikipedia.org/wiki/Grover%27s_algorithm)
- [Qiskit tutorial](https://qiskit.org/textbook/ch-algorithms/grover.html)

In classical computation assume we have a function $F:X\rightarrow \{0, 1\}$. 

If there is only one element $x\in X$ for which $F(x)=1$, we need $\mathcal{O}(|X|)$ checks to find it.

In quantum system we can define a function $F$ to mark desired states with a phase flip.

Using this we can build an algorithm to find $x$ in $\mathcal{O}(\sqrt{|X|})$.

Algorithm important steps:
1. Phase Oracle
2. Grover Diffusion
3. Number of iterations

## Constructing the Grover Oracle

See details in [qiskit texbook](https://qiskit.org/textbook/ch-algorithms/grover.html). It shows the idea, how to convert "Classical Function" $f$ into a function of a desired type. The mechanism we have there is called *Phase Kickback*.

**Kickback**: Controlling qubit (or register) accepts the phase change of $U$ eigenvalue (eigenphase).

How do we use the effect? We CAN construct "Phase Oracle" using "Classic Quantum Oracle".

This is Classical Oracle:

![](https://qiskit.org/textbook/ch-algorithms/images/grover_boolean_oracle.svg)

Initialize output qubit with $|-\rangle$ and ignore it!

![](https://qiskit.org/textbook/ch-algorithms/images/grover_phase_oracle.svg)

## Diffusor = Inversion over mean

**Idea #1**. How will the **mean value** of a dataset $X$ change, if we replace single $x_i$ with $-x_i$? If 2 values? If half?

## Shortest effect demo ever

1. Implement 2 methods.
2. Change $N$ to see how it affects the result.

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

canvas = np.ones((20, 100)) / (20 ** .5)
markers = np.zeros((20,))

N = 1

for i in range(N):
    markers[random.randrange(canvas.shape[0])] = 1

    
def invert(markers, vec):
    # invert those values, which are markers[i] != 0
    res = vec.copy()
    # TODO
    return res


def reflect_over_mean(vec):
    # reflect all values over mean value
    res = vec.copy()
    # TODO
    return res

for i in range(1, 100):
    inv = invert(markers, canvas[:, i-1])
    ref = reflect_over_mean(inv)
    canvas[:, i] = ref

# DRAWING FUNCTIONS
_xx, _yy = np.meshgrid(np.arange(canvas.shape[1]), np.arange(canvas.shape[0]))
x, y = _xx.ravel(), _yy.ravel()
heights = [canvas[j, i] for i, j in zip(x, y)]
bottom = np.zeros_like(heights)

fig = plt.figure(figsize=(30, 50))
ax1 = fig.add_subplot(121, projection='3d')
ax1.bar3d(x, y, 0, 1, 1, heights, shade=True, alpha=.9)
plt.show() 
plt.figure(figsize=(15, 3))
plt.imshow(canvas, cmap='gray')
plt.show()

Let's produce IoM in a matrix form.

Did you notice, that $v'_i = 2\bar{v} - v_i = 2\bar{v} - 1*v_i$?

Generally $v' = 2\bar{v} - v = 2*\mathbb{1}\bar{v} - Iv$

Let's expand:

$\bar{v} = \frac{1}{N}\sum_i{v_i} = \sum_i\frac{1}{N}{v_i}= \frac{1}{N}\mathbb{1}^Tv$

Ok, now:

$v' = 2\frac{1}{N}\mathbb{1}\mathbb{1}^Tv - Iv = (2\frac{1}{N}\mathbb{E} - I)v$

## Ok. How to we do such matrices?

In [ ]:
H = np.array([[1, 1], [1, -1]]) * (.5 ** .5)
X = np.array([[0, 1], [1, 0]])
CZ = np.eye((4))
CZ[-1, -1] = -1
print("H")
print(np.kron(H, H))
print("HX")
print(np.kron(X, X) @ np.kron(H, H))
print("HX(CZ)")
print(CZ @ np.kron(X, X) @ np.kron(H, H))
print("HX(CZ)XH")
print(np.kron(H, H) @ np.kron(X, X) @ CZ @ np.kron(X, X) @ np.kron(H, H))

$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\stateplus}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\stateminus}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\I}{ \mymatrix{rr}{1 & 0 \\ 0 & 1}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \newcommand{\pstate}[1]{ \lceil \mspace{-1mu} #1 \mspace{-1.5mu} \rfloor } $
$ \newcommand{\greenbit}[1] {\mathbf{{\color{green}#1}}} $
$ \newcommand{\bluebit}[1] {\mathbf{{\color{blue}#1}}} $
$ \newcommand{\redbit}[1] {\mathbf{{\color{red}#1}}} $
$ \newcommand{\brownbit}[1] {\mathbf{{\color{brown}#1}}} $
$ \newcommand{\blackbit}[1] {\mathbf{{\color{black}#1}}} $

### Mathematical derivation of the reflection by inversion (version 1) -- QBronze version

It is clear that query operators reflect the quantum state on the unit circle over $x$-axis.

On the other hand, the inversion operator reflects the quantum state on the unit circle over the line defined by the initial state, say $ \ket{u} $. This fact is not so obvious and we present here how to derive it. ($ \bra{u} $ is the conjugate transpose of the vector $ \ket{u} $.)

The initial quantum state is $ \ket{u} = \myvector{\frac{1}{\sqrt{N}} \\ \vdots \\ \frac{1}{\sqrt{N}}}$ and the inversion is a linear operator and represented by the matrix:

$$ D = 2 \mymatrix{ccc}{
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    \vdots & \ddots & \vdots \\
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    } 
- I . $$

Since $ \ket{u} \bra{u} = \mymatrix{ccc}{
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    \vdots & \ddots & \vdots \\
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    } $, we can represent $ D $ in terms of $ \ket{u} $ as $ D = 2 \ket{u} \bra{u} - I$.
    
Let our current quantum state be $a \ket{u} + b \ket{u^\perp}$, where $\ket{u^\perp}$ denotes the state, which is orthogonal (perpendicular) to $\ket{u}$. After applying $D$ to our current quantum state, we obtain

$$D \big(a \ket{u} + b \ket{u^\perp}\big) = \big(2 \ket{u} \bra{u} - I \big) \big(a \ket{u} + b \ket{u^\perp} \big) = a \big(2 \ket{u} \bra{u} \ket{u} - \ket{u} \big) + b \big(2 \ket{u} \bra{u} \ket{u^\perp} - \ket{u^\perp} \big). $$

To simplify this equation, we use the following two facts:
<ul>
    <li>$\bra{u} \ket{u} = 1$, because the inner product of a quantum state gives its length square, which is equal to 1;</li>
    <li>$\bra{u} \ket{u^\perp} = 0$, because the states are orthogonal to each other.</li>
</ul>

$$ a \big( 2 \ket{u} \bra{u} \ket{u} - \ket{u} \big) + b \big( 2 \ket{u} \bra{u} \ket{u^\perp} - \ket{u^\perp} \big) = a \big( 2 \ket{u} - \ket{u} \big) + b \big( 2 \ket{u} \cdot 0 - \ket{u^\perp} \big) = a \ket{u} - b \ket{u^\perp}. $$ 

As $D (a \ket{u} + b \ket{u^\perp}) = a \ket{u} - b \ket{u^\perp}$, we conclude that $D$ is a reflection over axis formed by the state $\ket{u}$.


### From QBronze (version 2)
Now let's try to understand why these gates are chosen. Let's recall the inversion operator:

$$ 2 \mymatrix{ccc}{
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    \vdots & \ddots & \vdots \\
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    } 
- I . $$


This operator is also called the <font color="blue"> diffusion operator</font>. 

Recall that the diffusion operator can be expressed as $D=2\ket{u}\bra{u}-I$ where $\ket{u}=H^{\otimes n}\ket{0^n}$ is the equal superposition vector. We will simply denote $\ket{0^n}$ by $\ket{\mathbf{0}}$.

- To start with let's express $D$ as follows:

\begin{align*}
D=2\ket{u}\bra{u}-I &= 2H^{\otimes n}\ket{\mathbf{0}}\bra{\mathbf{0}}H^{\otimes n}-I \\
&= 2H^{\otimes n}\ket{\mathbf{0}}\bra{\mathbf{0}}H^{\otimes n}-H^{\otimes n}H^{\otimes n}\\
&=H^{\otimes n} (2\ket{\mathbf{0}}\bra{\mathbf{0}}H^{\otimes n}-H^{\otimes n}) \\
&=H^{\otimes n} (2\ket{\mathbf{0}}\bra{\mathbf{0}}-I)H^{\otimes n}
\end{align*}

<font color="blue"><i>Looking at this expression, it is clear why we have H gates at the beginning and at the end</i>
    
- Now let us look at the effect of applying $2\ket{\mathbf{0}}\bra{\mathbf{0}}-I$ to any arbitrary state.

$(2\ket{\mathbf{0}}\bra{\mathbf{0}}-I) \ket{x} = 2\ket{\mathbf{0}}\braket{\mathbf{0}}{x}-\ket{x} .$

If $\ket{x}=\ket{\mathbf{0}}$, since $\braket{\mathbf{0}}{\mathbf{0}}=1$, then $2\ket{\mathbf{0}}\braket{\mathbf{0}}{\mathbf{0}}-\ket{\mathbf{0}} = 2\ket{\mathbf{0}}-\ket{\mathbf{0}} = \ket{\mathbf{0}}$.

If $\ket{x}\neq \ket{\mathbf{0}}$, since $\braket{\mathbf{0}}{x}=0$, then $2\ket{\mathbf{0}}\braket{\mathbf{0}}{x}-\ket{x}= 2\ket{\mathbf{0}}\cdot 0 -\ket{x} = -\ket{x}$.

Hence, the effect of $2\ket{\mathbf{0}}\bra{\mathbf{0}}-I$  is flipping the amplitude of any state except $\ket{\mathbf{0}}$.
    
- Now let's see how we can implement this operator. Let's define function $g$ as follows and let $U_g$ be the corresponding operator. 

\begin{align*}
g(x)&=0 &\mbox{ if $x$ is $\ket{\mathbf{0}}$ }\\
g(x)&=1 &\mbox{ otherwise},
\end{align*}



Let's set ancilla qubit to state $\ket{-}$ and apply operator $U_g$.
\begin{align*}
U_g \ket{x}\ket{-} &= (-1)^{g(x)} \ket{x} \ket{-}.
\end{align*}


Note that $U_g$ flips the amplitudes of the states other than $\ket{\mathbf{0}}$ and exactly implements $2\ket{\mathbf{0}}\bra{\mathbf{0}}-I$.

## The number of iterations (QBronze)

If there is a single marked element in a list of size $ N $, then $ \pi \dfrac{\sqrt{N}}{4} $ iterations can give the marked element with high probability.

If there are $k$ marked elements, then it is better to iterate $ \pi \dfrac{\sqrt{\frac{N}{k}}}{4} $ times.

If $k$ is unknown, then we can execute the algorithm with different iterations. One way of doing this is to iterate the algorithm  
<br>
$ \pi \dfrac{\sqrt{\frac{N}{1}}}{4}, \pi \dfrac{\sqrt{\frac{N}{2}}}{4}, \pi \dfrac{\sqrt{\frac{N}{4}}}{4}, \pi \dfrac{\sqrt{\frac{N}{8}}}{4}, \ldots $ times. 

The total number of iterations will still be proportional to $ \pi \dfrac{\sqrt{N}}{4} $: $ O \Big( \pi \dfrac{\sqrt{N}}{4} \Big) $.

# And now we implement

Implement Grover Search for a function $f(x) = (a \land \lnot b \land c)$

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit.circuit.library import XGate, ZGate

### Classical Oracle with 4 qubits

In [ ]:
cccx = XGate().control(3)
orreg = QuantumRegister(4)
oracle = QuantumCircuit(orreg, name="oracle")

# TODO complete the code to implement f(x) in an oracle form

oracle.draw()

### Mirror gate for 3 qubits

In [ ]:
ccz = ZGate().control(2)
mirreg = QuantumRegister(3)
mirror = QuantumCircuit(mirreg, name="mirror")

# TODO complete the code to implement diffusor

mirror.draw()

### Compose the algorithm
1. Prepare kickback qubit `qr[3]` in $|-\rangle$.
2. Compute the number of iterations.
3. Write the loop of oracles and mirrors. *NB oracle has 4 qubits, mirror is 3.*

In [ ]:
qr = QuantumRegister(4, "q")
cr = ClassicalRegister(3, "c")
qc = QuantumCircuit(qr, cr)

qc.h(qr[:3])
qc.barrier()

#TODO prepare qr[3] in |-> state.

qc.barrier()


from math import pi, ceil

N = 3
k = 1
iters = 0 # TODO
print(iters)

# TODO: write iterations `iters` times

qc.measure(qr[:3], cr)
print(qc.draw())

job = execute(qc, Aer.get_backend('qasm_simulator'), shots=10000)
counts = job.result().get_counts(qc)
plot_histogram(counts)